In [4]:
from dotenv import load_dotenv
import os
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')


In [14]:
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key=OPENAI_API_KEY,temperature=0.6)
# text="Who is the author of the book Harry Potter?"
# print(llm.predict(text))

In [17]:
from langchain import HuggingFaceHub
llm_huggingface=HuggingFaceHub(huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN, repo_id="TheBloke/Llama-2-7B-Chat-GGML",model_kwargs={"temperature":0.4,"max_length":264})

In [18]:
output=llm_huggingface.predict("Write a haiku about sadness")

print(output)

BadRequestError:  (Request ID: RgaKvPW9VYVLrwQR_yATB)

Bad request:
Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query.